##匯入資料

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model  import LogisticRegression
from IPython.display import clear_output
from sklearn.model_selection import KFold
shop_df = pd.read_csv('./online_shoppers_intention.csv')
shop_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12330 entries, 0 to 12329
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Administrative           12330 non-null  int64  
 1   Administrative_Duration  12330 non-null  float64
 2   Informational            12330 non-null  int64  
 3   Informational_Duration   12330 non-null  float64
 4   ProductRelated           12330 non-null  int64  
 5   ProductRelated_Duration  12330 non-null  float64
 6   BounceRates              12330 non-null  float64
 7   ExitRates                12330 non-null  float64
 8   PageValues               12330 non-null  float64
 9   SpecialDay               12330 non-null  float64
 10  Month                    12330 non-null  object 
 11  OperatingSystems         12330 non-null  int64  
 12  Browser                  12330 non-null  int64  
 13  Region                   12330 non-null  int64  
 14  TrafficType           

In [0]:
true = shop_df.loc[shop_df['Revenue']==True]
true.describe()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,OperatingSystems,Browser,Region,TrafficType
count,1908.000000,1908.000000,1908.000000,1908.000000,1908.000000,1908.000000,1908.000000,1908.000000,1908.000000,1908.000000,1908.000000,1908.000000,1908.000000,1908.000000
mean,3.393606,119.483244,0.786164,57.611427,48.210168,1876.209615,0.005117,0.019555,27.264518,0.023166,2.092767,2.453354,3.082285,4.021488
std,3.730527,201.114732,1.521176,171.619131,58.267365,2312.214392,0.012185,0.016463,35.191954,0.123521,0.929779,1.897042,2.407173,3.987140
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,15.000000,541.906250,0.000000,0.009521,3.641144,0.000000,2.000000,2.000000,1.000000,2.000000
50%,2.000000,52.366667,0.000000,0.000000,29.000000,1109.906250,0.000000,0.016000,16.758134,0.000000,2.000000,2.000000,2.000000,2.000000
75%,5.000000,151.083333,1.000000,19.000000,57.000000,2266.011310,0.006452,0.025000,38.897742,0.000000,2.000000,2.000000,4.000000,4.000000
max,26.000000,2086.750000,12.000000,1767.666667,534.000000,27009.859430,0.200000,0.200000,361.763742,1.000000,8.000000,13.000000,9.000000,20.000000


In [0]:
true[['SpecialDay','BounceRates','ExitRates','PageValues']].describe()

,SpecialDay,BounceRates,ExitRates,PageValues
count,1908.000000,1908.000000,1908.000000,1908.000000
mean,0.023166,0.005117,0.019555,27.264518
std,0.123521,0.012185,0.016463,35.191954
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.009521,3.641144
50%,0.000000,0.000000,0.016000,16.758134
75%,0.000000,0.006452,0.025000,38.897742
max,1.000000,0.200000,0.200000,361.763742


In [0]:
false = shop_df.loc[shop_df['Revenue']==False]
false.describe()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,OperatingSystems,Browser,Region,TrafficType
count,10422.000000,10422.000000,10422.000000,10422.000000,10422.000000,10422.000000,10422.000000,10422.000000,10422.000000,10422.000000,10422.000000,10422.000000,10422.000000,10422.000000
mean,2.117732,73.740111,0.451833,30.236237,28.714642,1069.987809,0.025317,0.047378,1.975998,0.068432,2.129726,2.339474,3.159278,4.078392
std,3.202481,171.017970,1.211582,133.909306,40.744717,1803.797757,0.051877,0.051231,9.072424,0.209054,0.907835,1.681790,2.400493,4.032220
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,6.000000,151.000000,0.000000,0.015560,0.000000,0.000000,2.000000,2.000000,1.000000,2.000000
50%,0.000000,0.000000,0.000000,0.000000,16.000000,510.190000,0.004255,0.028571,0.000000,0.000000,2.000000,2.000000,3.000000,2.000000
75%,3.000000,83.000000,0.000000,0.000000,35.000000,1331.816667,0.020000,0.053846,0.000000,0.000000,3.000000,2.000000,4.000000,4.000000
max,27.000000,3398.750000,24.000000,2549.375000,705.000000,63973.522230,0.200000,0.200000,246.758590,1.000000,8.000000,13.000000,9.000000,20.000000


In [0]:
false[['SpecialDay','BounceRates','ExitRates','PageValues']].describe()

,SpecialDay,BounceRates,ExitRates,PageValues
count,10422.000000,10422.000000,10422.000000,10422.000000
mean,0.068432,0.025317,0.047378,1.975998
std,0.209054,0.051877,0.051231,9.072424
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.015560,0.000000
50%,0.000000,0.004255,0.028571,0.000000
75%,0.000000,0.020000,0.053846,0.000000
max,1.000000,0.200000,0.200000,246.758590


看上方的圖表發現Revenue的true,false的資料裡相差最多的是

Administrative_Duration,Informational_Duration,ProductRelated_Duration,還有pagevalue

In [0]:
shoptrain_x = shop_df[['Administrative_Duration','Informational_Duration','ProductRelated_Duration']]
shoptrain_y = shop_df['Revenue']

In [0]:
kf = KFold(n_splits=10,                                # 設定 K 值
           random_state=1012,
           shuffle=True,
           )
kf.get_n_splits(shoptrain_x)   
train_acc_list = []                                   
valid_acc_list = []
for train_index, valid_index in kf.split(shoptrain_x):    
    train_x_split = shoptrain_x.iloc[train_index]         
    train_y_split = shoptrain_y.iloc[train_index]         
    valid_x_split = shoptrain_x.iloc[valid_index]         
    valid_y_split = shoptrain_y.iloc[valid_index]  
    mode2 = LogisticRegression(random_state=1012)
    mode2.fit(train_x_split,train_y_split)  
    train_pred_y = mode2.predict(train_x_split)       # 確認模型是否訓練成功
    train_acc = accuracy_score(train_y_split,         # 計算訓練資料準確度
                               train_pred_y)
    valid_pred_y = mode2.predict(valid_x_split)       # 驗證模型是否訓練成功
    valid_acc = accuracy_score(valid_y_split,         # 計算驗證資料準確度
                               valid_pred_y)
    train_acc_list.append(train_acc)
    valid_acc_list.append(valid_acc)     

In [0]:
print((
    'average train accuracy: {}\n' +
    '    min train accuracy: {}\n' +
    '    max train accuracy: {}\n' +
    'average valid accuracy: {}\n' +
    '    min valid accuracy: {}\n' +
    '    max valid accuracy: {}\n').format(
    np.mean(train_acc_list),                          # 輸出平均訓練準確度
    np.min(train_acc_list),                           # 輸出最低訓練準確度
    np.max(train_acc_list),                           # 輸出最高訓練準確度
    np.mean(valid_acc_list),                          # 輸出平均驗證準確度
    np.min(valid_acc_list),                           # 輸出最低驗證準確度
    np.max(valid_acc_list)                           # 輸出最高驗證準確度
    ))
    

average train accuracy: 0.844453455888979
    min train accuracy: 0.8413985761917635
    max train accuracy: 0.8460845273497342
average valid accuracy: 0.8444444444444444
    min valid accuracy: 0.8329278183292782
    max valid accuracy: 0.8710462287104623



一開始我先用logesticregression去predict revenue

trainx 是'Administrative_Duration','Informational_Duration','ProductRelated_Duration'

做出來的accuracy是

average train accuracy: 0.844453455888979

average valid accuracy: 0.8444444444444444



In [0]:
shoptrain_x = shop_df[['Administrative_Duration','Informational_Duration','ProductRelated_Duration','PageValues']]
shoptrain_y = shop_df['Revenue']


In [0]:
kf = KFold(n_splits=10,                                # 設定 K 值
           random_state=1012,
           shuffle=True,
           )
kf.get_n_splits(shoptrain_x)   
train_acc_list = []                                   
valid_acc_list = []
for train_index, valid_index in kf.split(shoptrain_x):    
    train_x_split = shoptrain_x.iloc[train_index]         
    train_y_split = shoptrain_y.iloc[train_index]         
    valid_x_split = shoptrain_x.iloc[valid_index]         
    valid_y_split = shoptrain_y.iloc[valid_index]  
    mode2 =RandomForestClassifier(random_state=1012)
    mode2.fit(train_x_split,train_y_split)  
    train_pred_y = mode2.predict(train_x_split)       # 確認模型是否訓練成功
    train_acc = accuracy_score(train_y_split,         # 計算訓練資料準確度
                               train_pred_y)
    valid_pred_y = mode2.predict(valid_x_split)       # 驗證模型是否訓練成功
    valid_acc = accuracy_score(valid_y_split,         # 計算驗證資料準確度
                               valid_pred_y)
    train_acc_list.append(train_acc)
    valid_acc_list.append(valid_acc)  

clear_output()

In [0]:
print((
    'average train accuracy: {}\n' +
    '    min train accuracy: {}\n' +
    '    max train accuracy: {}\n' +
    'average valid accuracy: {}\n' +
    '    min valid accuracy: {}\n' +
    '    max valid accuracy: {}\n').format(
    np.mean(train_acc_list),                          # 輸出平均訓練準確度
    np.min(train_acc_list),                           # 輸出最低訓練準確度
    np.max(train_acc_list),                           # 輸出最高訓練準確度
    np.mean(valid_acc_list),                          # 輸出平均驗證準確度
    np.min(valid_acc_list),                           # 輸出最低驗證準確度
    np.max(valid_acc_list)                           # 輸出最高驗證準確度
    ))
    

average train accuracy: 0.998630260430747
    min train accuracy: 0.9982878255384338
    max train accuracy: 0.998918626655853
average valid accuracy: 0.8776155717761556
    min valid accuracy: 0.8678021086780211
    max valid accuracy: 0.8848337388483374



後來我發現pagevalues影響是最大的加了pagevalues後再用Randomforest去跑得結果是

average train accuracy: 0.998630260430747

average valid accuracy: 0.8776155717761556

In [0]:
kf = KFold(n_splits=10,                                # 設定 K 值
           random_state=1012,
           shuffle=True,
           )
kf.get_n_splits(shoptrain_x)   
train_acc_list = []                                   
valid_acc_list = []
for train_index, valid_index in kf.split(shoptrain_x):    
    train_x_split = shoptrain_x.iloc[train_index]         
    train_y_split = shoptrain_y.iloc[train_index]         
    valid_x_split = shoptrain_x.iloc[valid_index]         
    valid_y_split = shoptrain_y.iloc[valid_index]  
    mode2 =RandomForestClassifier(max_depth=10,n_estimators=11,random_state=1012)
    mode2.fit(train_x_split,train_y_split)  
    train_pred_y = mode2.predict(train_x_split)       # 確認模型是否訓練成功
    train_acc = accuracy_score(train_y_split,         # 計算訓練資料準確度
                               train_pred_y)
    valid_pred_y = mode2.predict(valid_x_split)       # 驗證模型是否訓練成功
    valid_acc = accuracy_score(valid_y_split,         # 計算驗證資料準確度
                               valid_pred_y)
    train_acc_list.append(train_acc)
    valid_acc_list.append(valid_acc)  

clear_output()

In [0]:
print((
    'average train accuracy: {}\n' +
    '    min train accuracy: {}\n' +
    '    max train accuracy: {}\n' +
    'average valid accuracy: {}\n' +
    '    min valid accuracy: {}\n' +
    '    max valid accuracy: {}\n').format(
    np.mean(train_acc_list),                          # 輸出平均訓練準確度
    np.min(train_acc_list),                           # 輸出最低訓練準確度
    np.max(train_acc_list),                           # 輸出最高訓練準確度
    np.mean(valid_acc_list),                          # 輸出平均驗證準確度
    np.min(valid_acc_list),                           # 輸出最低驗證準確度
    np.max(valid_acc_list)                           # 輸出最高驗證準確度
    ))
    

average train accuracy: 0.9325132918806884
    min train accuracy: 0.9292601604037127
    max train accuracy: 0.9347571415697936
average valid accuracy: 0.8871046228710462
    min valid accuracy: 0.8807785888077859
    max valid accuracy: 0.8969991889699919



最終調完的結果是

average train accuracy: 0.9325132918806884

average valid accuracy: 0.8871046228710462

##Data Analytics

Choose a dataset

online_shoppers_intension

Analyze the data

我先把revenue是True的分一邊False的分一邊然後去看它裡面的資料哪些是相差最多的

我觀察的結果是Administrative_Duration,Informational_Duration,ProductRelated_Duration,還有pagevalue是差最多的

也就是說這些是很有可能影響revenue是True還是False的關鍵

所以我用這些資料去當trainx去predict revenue

Define a reasonable problem

預測reveune是True還是False

explain how you improveed your results step-by-step

一開始我先把各個我認為可能是會有影響的點都當trainx用Logesticregression跑跑看

發現除了pagevalues以外大概都是0.84左右

有pagevalues的話大概是0.86左右

然後我把除了pagevalues以外資料全部丟進去所得到的結果是

average train accuracy: 0.844453455888979

average valid accuracy: 0.8444444444444444

所以的確pagevalues是最影響他判斷的資料

然後因為之前的作業Randomforest跑得比logesticregression還要好一點

所以我就用Randomforest去跑跑所有可能會有影響的資料得到的結果是

average train accuracy: 0.998630260430747

average valid accuracy: 0.8776155717761556

然後因為上個作業也有遇過類似overfitting的例子，那時我是調了max_depth,n_estimators所以這次我也調調看這兩個參數果然overfitting解決了而且valid aaccuracy也提升了得到的結果是

average train accuracy: 0.9325132918806884

average valid accuracy: 0.8871046228710462
